# Step 1: Import Data & Basic Cleaning

In [ ]:
# By: Atiwat Rachatawarn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_inbound = pd.read_csv('./Inbound.csv')
df_material = pd.read_csv('./MaterialMaster.csv')

In [ ]:
# Make all column uppercase for consistency

df_inbound.columns = df_inbound.columns.str.upper()
df_material.columns = df_material.columns.str.upper()

# Step 2: Feature Engineering, Add New Columns

In [ ]:
# New DF to keep track of temporary stored material.
df_temp = pd.DataFrame()

# Assume one container can hold up to this much in MT (changable to accomodate available containers).
container_capacity = 80
# Assume maximum allowed level for ceiling height/safety reasons. Adjustable.
max_level = 3

In [ ]:
# Grab rows from df_inbound to use as example for incoming material

df_incoming = df_inbound[df_inbound['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE'].sample(n=6)
df_incoming

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT
4631,2024/08/09,SINGAPORE-WAREHOUSE,MAT-0234,24.750
5149,2024/10/05,SINGAPORE-WAREHOUSE,MAT-0051,21.250
117,2024/10/15,SINGAPORE-WAREHOUSE,MAT-0171,24.750
5616,2024/10/08,SINGAPORE-WAREHOUSE,MAT-0172,13.750
2133,2024/04/02,SINGAPORE-WAREHOUSE,MAT-0263,15.125
1399,2024/03/03,SINGAPORE-WAREHOUSE,MAT-0234,4.125


In [ ]:
import pandas as pd

def add_incoming_to_temp(df_temp: pd.DataFrame,
                         df_incoming: pd.DataFrame,
                         df_material: pd.DataFrame) -> pd.DataFrame:
    """
    Adds incoming rows to the temporary DataFrame with STACK_ID, STACK_LEVEL,
    and a single SHELF_LIFE_IN_MONTH column (looked up from df_material).

    Args:
      df_temp:     Existing temporary DataFrame.
      df_incoming: New rows to append (may not have STACK_ID/STACK_LEVEL).
      df_material: DataFrame with ['MATERIAL_NAME','SHELF_LIFE_IN_MONTH', ...].

    Returns:
      The updated temporary DataFrame.
    """
    # 1) Ensure the incoming rows have the stack columns
    for col in ('STACK_ID','STACK_LEVEL'):
        if col not in df_incoming.columns:
            df_incoming[col] = None

    # 2) Append incoming rows
    updated = pd.concat([df_temp, df_incoming], ignore_index=True)

    # 3) Build a lookup series: MATERIAL_NAME -> SHELF_LIFE_IN_MONTH
    shelf_lookup = df_material.set_index('MATERIAL_NAME')['SHELF_LIFE_IN_MONTH']

    # 4) Map it into the updated DataFrame (will create one clean column)
    updated['SHELF_LIFE_IN_MONTH'] = updated['MATERIAL_NAME'].map(shelf_lookup)

    return updated


In [ ]:
# Now, the df_temp looks like this, lacking only stacking order
df_temp = add_incoming_to_temp(df_temp, df_incoming, df_material)
df_temp

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT,STACK_ID,STACK_LEVEL,SHELF_LIFE_IN_MONTH
0,2024/08/09,SINGAPORE-WAREHOUSE,MAT-0234,24.750,None,None,5
1,2024/10/05,SINGAPORE-WAREHOUSE,MAT-0051,21.250,None,None,8
2,2024/10/15,SINGAPORE-WAREHOUSE,MAT-0171,24.750,None,None,6
3,2024/10/08,SINGAPORE-WAREHOUSE,MAT-0172,13.750,None,None,6
4,2024/04/02,SINGAPORE-WAREHOUSE,MAT-0263,15.125,None,None,5
5,2024/03/03,SINGAPORE-WAREHOUSE,MAT-0234,4.125,None,None,5


# Step 3: Calculate and Assign Stacks Order

In [ ]:
import pandas as pd
import uuid

def assign_stacks(df: pd.DataFrame,
                  container_capacity: float,
                  max_level: int) -> pd.DataFrame:
    """
    df must have columns:
      - NET_QUANTITY_MT (float)
      - SHELF_LIFE_IN_MONTH (int)
      - STACK_ID (object / nullable)
      - STACK_LEVEL (int / nullable)

    Returns a new DataFrame with STACK_ID and STACK_LEVEL filled in
    for the previously-unassigned rows, ensuring:
      * Each stack's TOTAL weight does not exceed container_capacity.
      * Shelf life ordering: higher shelf life at lower levels.
      * Max levels capped at max_level.
    """
    df = df.copy()

    # 1) Build current stacks as lists of dicts
    stacks = {}  # stack_id -> list of {'weight','shelf','idx'}
    for idx, row in df.dropna(subset=['STACK_ID','STACK_LEVEL']).iterrows():
        sid = row['STACK_ID']
        w = float(row['NET_QUANTITY_MT'])
        shelf = int(row['SHELF_LIFE_IN_MONTH'])
        stacks.setdefault(sid, []).append({'weight': w, 'shelf': shelf, 'idx': idx})

    # Sort existing stacks by descending shelf and reassign levels
    for sid, items in stacks.items():
        items.sort(key=lambda x: -x['shelf'])
        for lvl, e in enumerate(items, start=1):
            df.at[e['idx'], 'STACK_LEVEL'] = lvl

    # 2) Prepare ID generator
    existing_ids = set(stacks.keys())
    def next_id():
        while True:
            u = str(uuid.uuid4())
            if u not in existing_ids:
                existing_ids.add(u)
                return u

    # 3) Assign unstacked containers
    for idx, row in df[df['STACK_ID'].isna()].iterrows():
        w_new = float(row['NET_QUANTITY_MT'])
        shelf_new = int(row['SHELF_LIFE_IN_MONTH'])
        placed = False

        # Try fitting into existing stacks
        for sid, items in stacks.items():
            if len(items) >= max_level:
                continue
            # Build candidate with new container
            candidate = items + [{'weight': w_new, 'shelf': shelf_new, 'idx': idx}]
            # Sort by descending shelf life
            candidate.sort(key=lambda x: -x['shelf'])
            # Check total weight limit
            total_weight = sum(e['weight'] for e in candidate)
            if total_weight > container_capacity:
                continue
            # Fits! commit
            stacks[sid] = candidate
            for lvl, e in enumerate(candidate, start=1):
                df.at[e['idx'], 'STACK_ID'] = sid
                df.at[e['idx'], 'STACK_LEVEL'] = lvl
            placed = True
            break

        # Start new stack if not placed
        if not placed:
            new_sid = next_id()
            stacks[new_sid] = [{'weight': w_new, 'shelf': shelf_new, 'idx': idx}]
            df.at[idx, 'STACK_ID'] = new_sid
            df.at[idx, 'STACK_LEVEL'] = 1

    return df


In [ ]:
df_temp = assign_stacks(df_temp, container_capacity, max_level)
df_temp

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT,STACK_ID,STACK_LEVEL,SHELF_LIFE_IN_MONTH
0,2024/08/09,SINGAPORE-WAREHOUSE,MAT-0234,24.750,bb32e36f-73e7-4492-a92a-e502d95cd107,3,5
1,2024/10/05,SINGAPORE-WAREHOUSE,MAT-0051,21.250,bb32e36f-73e7-4492-a92a-e502d95cd107,1,8
2,2024/10/15,SINGAPORE-WAREHOUSE,MAT-0171,24.750,bb32e36f-73e7-4492-a92a-e502d95cd107,2,6
3,2024/10/08,SINGAPORE-WAREHOUSE,MAT-0172,13.750,e182e119-a7f3-4f72-a85f-965b0bec0076,1,6
4,2024/04/02,SINGAPORE-WAREHOUSE,MAT-0263,15.125,e182e119-a7f3-4f72-a85f-965b0bec0076,2,5
5,2024/03/03,SINGAPORE-WAREHOUSE,MAT-0234,4.125,e182e119-a7f3-4f72-a85f-965b0bec0076,3,5


# Step 4: Summarize and Find Useful Conclusions

In [ ]:
# Summary of each stack

df_stack_info = df_temp.groupby('STACK_ID').agg(
    highest_level=('STACK_LEVEL', 'max'),
    total_weight=('NET_QUANTITY_MT', 'sum')
).reset_index()

df_stack_info

,STACK_ID,highest_level,total_weight
0,bb32e36f-73e7-4492-a92a-e502d95cd107,3,70.75
1,e182e119-a7f3-4f72-a85f-965b0bec0076,3,33.00


In [ ]:
# Find Stack Order for a given stack

target_stack_id = df_stack_info.at[0, 'STACK_ID']

filtered_df_temp = df_temp[df_temp['STACK_ID'] == target_stack_id].sort_values(by='STACK_LEVEL', ascending=False)
filtered_df_temp

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT,STACK_ID,STACK_LEVEL,SHELF_LIFE_IN_MONTH
0,2024/08/09,SINGAPORE-WAREHOUSE,MAT-0234,24.75,bb32e36f-73e7-4492-a92a-e502d95cd107,3,5
2,2024/10/15,SINGAPORE-WAREHOUSE,MAT-0171,24.75,bb32e36f-73e7-4492-a92a-e502d95cd107,2,6
1,2024/10/05,SINGAPORE-WAREHOUSE,MAT-0051,21.25,bb32e36f-73e7-4492-a92a-e502d95cd107,1,8
